In [1]:
#Import thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objs as go
from plotly.subplots import make_subplots

6:4

In [2]:
# Đọc dữ liệu
def read_data(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = df['Date'].str.strip()  # Xóa khoảng trắng
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')  # Xử lý ngày tháng với định dạng hỗn hợp
    return df

# Chia dữ liệu thành tập huấn luyện và kiểm tra
def split_data(df, test_size):
    train, test = train_test_split(df, test_size=test_size, shuffle=False)
    return train, test

# Huấn luyện mô hình
def train_models(train, predict_days, price_column='Price'):
    X_train = np.arange(len(train)).reshape(-1, 1)
    y_train = train[price_column]

    # Mô hình 1: RandomForestRegressor
    model1 = RandomForestRegressor()
    model1.fit(X_train, y_train)

    # Mô hình 2: BaggingRegressor với LinearRegression và GridSearchCV
    param_grid = {
        'n_estimators': [10, 20, 50],
        'max_samples': [0.5, 0.7, 1.0],
        'max_features': [0.5, 0.7, 1.0]
    }
    base_estimator = LinearRegression()
    model2 = BaggingRegressor(estimator=base_estimator)

    grid_search = GridSearchCV(estimator=model2, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
    grid_search.fit(X_train, y_train)
    best_model2 = grid_search.best_estimator_

    # Dự đoán cho các ngày trong tương lai
    start_idx = len(train)
    pred_indices = np.arange(start_idx, start_idx + predict_days).reshape(-1, 1)
    preds1 = model1.predict(pred_indices)
    preds2 = best_model2.predict(pred_indices)

    return preds1, preds2

# Đánh giá mô hình
def evaluate_model(true_values, predictions):
    min_length = min(len(true_values), len(predictions))
    true_values = true_values[:min_length]
    predictions = predictions[:min_length]

    rmse = np.sqrt(mean_squared_error(true_values, predictions))
    mae = mean_absolute_error(true_values, predictions)
    mape = np.mean(np.abs((true_values - predictions) / true_values)) * 100
    return rmse, mae, mape

# Vẽ đồ thị bằng Plotly
def plot_results(train, test, preds_30, preds_60, preds_90, dates_30, dates_60, dates_90, stock_name, model_name):
    fig = make_subplots(rows=1, cols=1)

    # Dữ liệu hiển thị đồ thị
    train_to_show = train
    test_to_show = test

    # Thêm dữ liệu huấn luyện
    fig.add_trace(go.Scatter(x=train_to_show.index, y=train_to_show['Price'], mode='lines', name='Training Data', line=dict(color='orange')), row=1, col=1)

    # Thêm dữ liệu kiểm tra
    fig.add_trace(go.Scatter(x=test_to_show.index, y=test_to_show['Price'], mode='lines', name='Test Data', line=dict(color='cyan')), row=1, col=1)

    # Thêm dữ liệu dự đoán 30 ngày
    fig.add_trace(go.Scatter(x=dates_30, y=preds_30, mode='lines', name=f'{model_name} Predictions 30 Days', line=dict(color='green')), row=1, col=1)

    # Thêm dữ liệu dự đoán 60 ngày
    fig.add_trace(go.Scatter(x=dates_60, y=preds_60, mode='lines', name=f'{model_name} Predictions 60 Days', line=dict(color='purple')), row=1, col=1)

    # Thêm dữ liệu dự đoán 90 ngày
    fig.add_trace(go.Scatter(x=dates_90, y=preds_90, mode='lines', name=f'{model_name} Predictions 90 Days', line=dict(color='red')), row=1, col=1)

    fig.update_layout(title=f'{model_name} Price Prediction for Next 30, 60, and 90 Days - {stock_name}',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      template="plotly_white",
                       legend=dict(orientation="v", x=0, y=1),
                      legend_title="Legend"
                       )
    fig.show()

# Xử lý và dự đoán cho từng mã cổ phiếu
def process_stock(file_path, stock_name, ratios):
    df = read_data(file_path)
    df.set_index('Date', inplace=True)
    price_column = 'Price'  # Cột chứa giá cổ phiếu

    end_date = df.index[-1]  # Ngày cuối cùng của dữ liệu huấn luyện

    # Các mốc thời gian cụ thể
    dates_30 = pd.date_range(start=end_date + pd.Timedelta(days=1), end='2024-07-01', freq='D')[:-1]
    dates_60 = pd.date_range(start='2024-07-01', end='2024-08-01', freq='D')[:-1]
    dates_90 = pd.date_range(start='2024-08-01', end='2024-09-01', freq='D')[:-1]

    for ratio in ratios:
        train, test = split_data(df, test_size=ratio)
        preds1_30, preds2_30 = train_models(train, len(dates_30), price_column)
        preds1_60, preds2_60 = train_models(train, len(dates_60), price_column)
        preds1_90, preds2_90 = train_models(train, len(dates_90), price_column)

        # Đánh giá cho từng khoảng thời gian
        rmse1_30, mae1_30, mape1_30 = evaluate_model(test[price_column][:len(dates_30)], preds1_30)
        rmse2_30, mae2_30, mape2_30 = evaluate_model(test[price_column][:len(dates_30)], preds2_30)

        rmse1_60, mae1_60, mape1_60 = evaluate_model(test[price_column][len(dates_30):len(dates_30) + len(dates_60)], preds1_60)
        rmse2_60, mae2_60, mape2_60 = evaluate_model(test[price_column][len(dates_30):len(dates_30) + len(dates_60)], preds2_60)

        rmse1_90, mae1_90, mape1_90 = evaluate_model(test[price_column][len(dates_30) + len(dates_60):len(dates_30) + len(dates_60) + len(dates_90)], preds1_90)
        rmse2_90, mae2_90, mape2_90 = evaluate_model(test[price_column][len(dates_30) + len(dates_60):len(dates_30) + len(dates_60) + len(dates_90)], preds2_90)

        print(f'RandomForest - Ratio {1-ratio:.0%}:{ratio:.0%} - 30 Days - RMSE: {rmse1_30:.2f}, MAE: {mae1_30:.2f}, MAPE: {mape1_30:.2f}%')
        print(f'Bagging/LinearReg - Ratio {1-ratio:.0%}:{ratio:.0%} - 30 Days - RMSE: {rmse2_30:.2f}, MAE: {mae2_30:.2f}, MAPE: {mape2_30:.2f}%')
        print(f'RandomForest - Ratio {1-ratio:.0%}:{ratio:.0%} - 60 Days - RMSE: {rmse1_60:.2f}, MAE: {mae1_60:.2f}, MAPE: {mape1_60:.2f}%')
        print(f'Bagging/LinearReg - Ratio {1-ratio:.0%}:{ratio:.0%} - 60 Days - RMSE: {rmse2_60:.2f}, MAE: {mae2_60:.2f}, MAPE: {mape2_60:.2f}%')
        print(f'RandomForest - Ratio {1-ratio:.0%}:{ratio:.0%} - 90 Days - RMSE: {rmse1_90:.2f}, MAE: {mae1_90:.2f}, MAPE: {mape1_90:.2f}%')
        print(f'Bagging/LinearReg - Ratio {1-ratio:.0%}:{ratio:.0%} - 90 Days - RMSE: {rmse2_90:.2f}, MAE: {mae2_90:.2f}, MAPE: {mape2_90:.2f}%')

        # Vẽ kết quả của mô hình RandomForest
        plot_results(train, test, preds1_30, preds1_60, preds1_90, dates_30, dates_60, dates_90, stock_name, 'RandomForest')

        # Vẽ kết quả của mô hình Bagging/LinearRegression
        plot_results(train, test, preds2_30, preds2_60, preds2_90, dates_30, dates_60, dates_90, stock_name, 'Bagging/LR')

# Tỉ lệ train:test
ratios = [0.4]

# Dự đoán cho từng mã cổ phiếu
process_stock('DataSBS.csv', 'SBS', ratios)

RandomForest - Ratio 60%:40% - 30 Days - RMSE: 2811.22, MAE: 2190.34, MAPE: 20.59%
Bagging/LinearReg - Ratio 60%:40% - 30 Days - RMSE: 3097.07, MAE: 2441.39, MAPE: 22.88%
RandomForest - Ratio 60%:40% - 60 Days - RMSE: 5468.42, MAE: 5409.90, MAPE: 61.48%
Bagging/LinearReg - Ratio 60%:40% - 60 Days - RMSE: 5510.58, MAE: 5442.89, MAPE: 61.89%
RandomForest - Ratio 60%:40% - 90 Days - RMSE: 5253.99, MAE: 5099.45, MAPE: 57.56%
Bagging/LinearReg - Ratio 60%:40% - 90 Days - RMSE: 5368.48, MAE: 5199.62, MAPE: 58.75%


7:3

In [3]:
# Đọc dữ liệu
def read_data(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = df['Date'].str.strip()  # Xóa khoảng trắng
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')  # Xử lý ngày tháng với định dạng hỗn hợp
    return df

# Chia dữ liệu thành tập huấn luyện và kiểm tra
def split_data(df, test_size):
    train, test = train_test_split(df, test_size=test_size, shuffle=False)
    return train, test

# Huấn luyện mô hình
def train_models(train, predict_days, price_column='Price'):
    X_train = np.arange(len(train)).reshape(-1, 1)
    y_train = train[price_column]

    # Mô hình 1: RandomForestRegressor
    model1 = RandomForestRegressor()
    model1.fit(X_train, y_train)

    # Mô hình 2: BaggingRegressor với LinearRegression và GridSearchCV
    param_grid = {
        'n_estimators': [10, 20, 50],
        'max_samples': [0.5, 0.7, 1.0],
        'max_features': [0.5, 0.7, 1.0]
    }
    base_estimator = LinearRegression()
    model2 = BaggingRegressor(estimator=base_estimator)

    grid_search = GridSearchCV(estimator=model2, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
    grid_search.fit(X_train, y_train)
    best_model2 = grid_search.best_estimator_

    # Dự đoán cho các ngày trong tương lai
    start_idx = len(train)
    pred_indices = np.arange(start_idx, start_idx + predict_days).reshape(-1, 1)
    preds1 = model1.predict(pred_indices)
    preds2 = best_model2.predict(pred_indices)

    return preds1, preds2

# Đánh giá mô hình
def evaluate_model(true_values, predictions):
    min_length = min(len(true_values), len(predictions))
    true_values = true_values[:min_length]
    predictions = predictions[:min_length]

    rmse = np.sqrt(mean_squared_error(true_values, predictions))
    mae = mean_absolute_error(true_values, predictions)
    mape = np.mean(np.abs((true_values - predictions) / true_values)) * 100
    return rmse, mae, mape

# Vẽ đồ thị bằng Plotly
def plot_results(train, test, preds_30, preds_60, preds_90, dates_30, dates_60, dates_90, stock_name, model_name):
    fig = make_subplots(rows=1, cols=1)

    # Dữ liệu hiển thị đồ thị
    train_to_show = train
    test_to_show = test

    # Thêm dữ liệu huấn luyện
    fig.add_trace(go.Scatter(x=train_to_show.index, y=train_to_show['Price'], mode='lines', name='Training Data', line=dict(color='orange')), row=1, col=1)

    # Thêm dữ liệu kiểm tra
    fig.add_trace(go.Scatter(x=test_to_show.index, y=test_to_show['Price'], mode='lines', name='Test Data', line=dict(color='cyan')), row=1, col=1)

    # Thêm dữ liệu dự đoán 30 ngày
    fig.add_trace(go.Scatter(x=dates_30, y=preds_30, mode='lines', name=f'{model_name} Predictions 30 Days', line=dict(color='green')), row=1, col=1)

    # Thêm dữ liệu dự đoán 60 ngày
    fig.add_trace(go.Scatter(x=dates_60, y=preds_60, mode='lines', name=f'{model_name} Predictions 60 Days', line=dict(color='purple')), row=1, col=1)

    # Thêm dữ liệu dự đoán 90 ngày
    fig.add_trace(go.Scatter(x=dates_90, y=preds_90, mode='lines', name=f'{model_name} Predictions 90 Days', line=dict(color='red')), row=1, col=1)

    fig.update_layout(title=f'{model_name} Price Prediction for Next 30, 60, and 90 Days - {stock_name}',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      template="plotly_white",
                       legend=dict(orientation="v", x=0, y=1),
                      legend_title="Legend"
                       )
    fig.show()

# Xử lý và dự đoán cho từng mã cổ phiếu
def process_stock(file_path, stock_name, ratios):
    df = read_data(file_path)
    df.set_index('Date', inplace=True)
    price_column = 'Price'  # Cột chứa giá cổ phiếu

    end_date = df.index[-1]  # Ngày cuối cùng của dữ liệu huấn luyện

    # Các mốc thời gian cụ thể
    dates_30 = pd.date_range(start=end_date + pd.Timedelta(days=1), end='2024-07-01', freq='D')[:-1]
    dates_60 = pd.date_range(start='2024-07-01', end='2024-08-01', freq='D')[:-1]
    dates_90 = pd.date_range(start='2024-08-01', end='2024-09-01', freq='D')[:-1]

    for ratio in ratios:
        train, test = split_data(df, test_size=ratio)
        preds1_30, preds2_30 = train_models(train, len(dates_30), price_column)
        preds1_60, preds2_60 = train_models(train, len(dates_60), price_column)
        preds1_90, preds2_90 = train_models(train, len(dates_90), price_column)

        # Đánh giá cho từng khoảng thời gian
        rmse1_30, mae1_30, mape1_30 = evaluate_model(test[price_column][:len(dates_30)], preds1_30)
        rmse2_30, mae2_30, mape2_30 = evaluate_model(test[price_column][:len(dates_30)], preds2_30)

        rmse1_60, mae1_60, mape1_60 = evaluate_model(test[price_column][len(dates_30):len(dates_30) + len(dates_60)], preds1_60)
        rmse2_60, mae2_60, mape2_60 = evaluate_model(test[price_column][len(dates_30):len(dates_30) + len(dates_60)], preds2_60)

        rmse1_90, mae1_90, mape1_90 = evaluate_model(test[price_column][len(dates_30) + len(dates_60):len(dates_30) + len(dates_60) + len(dates_90)], preds1_90)
        rmse2_90, mae2_90, mape2_90 = evaluate_model(test[price_column][len(dates_30) + len(dates_60):len(dates_30) + len(dates_60) + len(dates_90)], preds2_90)

        print(f'RandomForest - Ratio {1-ratio:.0%}:{ratio:.0%} - 30 Days - RMSE: {rmse1_30:.2f}, MAE: {mae1_30:.2f}, MAPE: {mape1_30:.2f}%')
        print(f'Bagging/LinearReg - Ratio {1-ratio:.0%}:{ratio:.0%} - 30 Days - RMSE: {rmse2_30:.2f}, MAE: {mae2_30:.2f}, MAPE: {mape2_30:.2f}%')
        print(f'RandomForest - Ratio {1-ratio:.0%}:{ratio:.0%} - 60 Days - RMSE: {rmse1_60:.2f}, MAE: {mae1_60:.2f}, MAPE: {mape1_60:.2f}%')
        print(f'Bagging/LinearReg - Ratio {1-ratio:.0%}:{ratio:.0%} - 60 Days - RMSE: {rmse2_60:.2f}, MAE: {mae2_60:.2f}, MAPE: {mape2_60:.2f}%')
        print(f'RandomForest - Ratio {1-ratio:.0%}:{ratio:.0%} - 90 Days - RMSE: {rmse1_90:.2f}, MAE: {mae1_90:.2f}, MAPE: {mape1_90:.2f}%')
        print(f'Bagging/LinearReg - Ratio {1-ratio:.0%}:{ratio:.0%} - 90 Days - RMSE: {rmse2_90:.2f}, MAE: {mae2_90:.2f}, MAPE: {mape2_90:.2f}%')

        # Vẽ kết quả của mô hình RandomForest
        plot_results(train, test, preds1_30, preds1_60, preds1_90, dates_30, dates_60, dates_90, stock_name, 'RandomForest')

        # Vẽ kết quả của mô hình Bagging/LinearRegression
        plot_results(train, test, preds2_30, preds2_60, preds2_90, dates_30, dates_60, dates_90, stock_name, 'Bagging/LR')

# Tỉ lệ train:test
ratios = [0.3]

# Dự đoán cho từng mã cổ phiếu
process_stock('DataSBS.csv', 'SBS', ratios)

RandomForest - Ratio 70%:30% - 30 Days - RMSE: 1812.65, MAE: 1623.93, MAPE: 39.24%
Bagging/LinearReg - Ratio 70%:30% - 30 Days - RMSE: 9675.69, MAE: 9634.49, MAPE: 217.03%
RandomForest - Ratio 70%:30% - 60 Days - RMSE: 1683.47, MAE: 1307.45, MAPE: 33.25%
Bagging/LinearReg - Ratio 70%:30% - 60 Days - RMSE: 9303.49, MAE: 9252.12, MAPE: 200.88%
RandomForest - Ratio 70%:30% - 90 Days - RMSE: 1096.42, MAE: 1077.32, MAPE: 20.86%
Bagging/LinearReg - Ratio 70%:30% - 90 Days - RMSE: 8901.99, MAE: 8898.34, MAPE: 171.24%


8:2

In [4]:
# Đọc dữ liệu
def read_data(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = df['Date'].str.strip()  # Xóa khoảng trắng
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')  # Xử lý ngày tháng với định dạng hỗn hợp
    return df

# Chia dữ liệu thành tập huấn luyện và kiểm tra
def split_data(df, test_size):
    train, test = train_test_split(df, test_size=test_size, shuffle=False)
    return train, test

# Huấn luyện mô hình
def train_models(train, predict_days, price_column='Price'):
    X_train = np.arange(len(train)).reshape(-1, 1)
    y_train = train[price_column]

    # Mô hình 1: RandomForestRegressor
    model1 = RandomForestRegressor()
    model1.fit(X_train, y_train)

    # Mô hình 2: BaggingRegressor với LinearRegression và GridSearchCV
    param_grid = {
        'n_estimators': [10, 20, 50],
        'max_samples': [0.5, 0.7, 1.0],
        'max_features': [0.5, 0.7, 1.0]
    }
    base_estimator = LinearRegression()
    model2 = BaggingRegressor(estimator=base_estimator)

    grid_search = GridSearchCV(estimator=model2, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
    grid_search.fit(X_train, y_train)
    best_model2 = grid_search.best_estimator_

    # Dự đoán cho các ngày trong tương lai
    start_idx = len(train)
    pred_indices = np.arange(start_idx, start_idx + predict_days).reshape(-1, 1)
    preds1 = model1.predict(pred_indices)
    preds2 = best_model2.predict(pred_indices)

    return preds1, preds2

# Đánh giá mô hình
def evaluate_model(true_values, predictions):
    min_length = min(len(true_values), len(predictions))
    true_values = true_values[:min_length]
    predictions = predictions[:min_length]

    rmse = np.sqrt(mean_squared_error(true_values, predictions))
    mae = mean_absolute_error(true_values, predictions)
    mape = np.mean(np.abs((true_values - predictions) / true_values)) * 100
    return rmse, mae, mape

# Vẽ đồ thị bằng Plotly
def plot_results(train, test, preds_30, preds_60, preds_90, dates_30, dates_60, dates_90, stock_name, model_name):
    fig = make_subplots(rows=1, cols=1)

    # Dữ liệu hiển thị đồ thị
    train_to_show = train
    test_to_show = test

    # Thêm dữ liệu huấn luyện
    fig.add_trace(go.Scatter(x=train_to_show.index, y=train_to_show['Price'], mode='lines', name='Training Data', line=dict(color='orange')), row=1, col=1)

    # Thêm dữ liệu kiểm tra
    fig.add_trace(go.Scatter(x=test_to_show.index, y=test_to_show['Price'], mode='lines', name='Test Data', line=dict(color='cyan')), row=1, col=1)

    # Thêm dữ liệu dự đoán 30 ngày
    fig.add_trace(go.Scatter(x=dates_30, y=preds_30, mode='lines', name=f'{model_name} Predictions 30 Days', line=dict(color='green')), row=1, col=1)

    # Thêm dữ liệu dự đoán 60 ngày
    fig.add_trace(go.Scatter(x=dates_60, y=preds_60, mode='lines', name=f'{model_name} Predictions 60 Days', line=dict(color='purple')), row=1, col=1)

    # Thêm dữ liệu dự đoán 90 ngày
    fig.add_trace(go.Scatter(x=dates_90, y=preds_90, mode='lines', name=f'{model_name} Predictions 90 Days', line=dict(color='red')), row=1, col=1)

    fig.update_layout(title=f'{model_name} Price Prediction for Next 30, 60, and 90 Days - {stock_name}',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      template="plotly_white",
                       legend=dict(orientation="v", x=0, y=1),
                      legend_title="Legend"
                       )
    fig.show()

# Xử lý và dự đoán cho từng mã cổ phiếu
def process_stock(file_path, stock_name, ratios):
    df = read_data(file_path)
    df.set_index('Date', inplace=True)
    price_column = 'Price'  # Cột chứa giá cổ phiếu

    end_date = df.index[-1]  # Ngày cuối cùng của dữ liệu huấn luyện

    # Các mốc thời gian cụ thể
    dates_30 = pd.date_range(start=end_date + pd.Timedelta(days=1), end='2024-07-01', freq='D')[:-1]
    dates_60 = pd.date_range(start='2024-07-01', end='2024-08-01', freq='D')[:-1]
    dates_90 = pd.date_range(start='2024-08-01', end='2024-09-01', freq='D')[:-1]

    for ratio in ratios:
        train, test = split_data(df, test_size=ratio)
        preds1_30, preds2_30 = train_models(train, len(dates_30), price_column)
        preds1_60, preds2_60 = train_models(train, len(dates_60), price_column)
        preds1_90, preds2_90 = train_models(train, len(dates_90), price_column)

        # Đánh giá cho từng khoảng thời gian
        rmse1_30, mae1_30, mape1_30 = evaluate_model(test[price_column][:len(dates_30)], preds1_30)
        rmse2_30, mae2_30, mape2_30 = evaluate_model(test[price_column][:len(dates_30)], preds2_30)

        rmse1_60, mae1_60, mape1_60 = evaluate_model(test[price_column][len(dates_30):len(dates_30) + len(dates_60)], preds1_60)
        rmse2_60, mae2_60, mape2_60 = evaluate_model(test[price_column][len(dates_30):len(dates_30) + len(dates_60)], preds2_60)

        rmse1_90, mae1_90, mape1_90 = evaluate_model(test[price_column][len(dates_30) + len(dates_60):len(dates_30) + len(dates_60) + len(dates_90)], preds1_90)
        rmse2_90, mae2_90, mape2_90 = evaluate_model(test[price_column][len(dates_30) + len(dates_60):len(dates_30) + len(dates_60) + len(dates_90)], preds2_90)

        print(f'RandomForest - Ratio {1-ratio:.0%}:{ratio:.0%} - 30 Days - RMSE: {rmse1_30:.2f}, MAE: {mae1_30:.2f}, MAPE: {mape1_30:.2f}%')
        print(f'Bagging/LinearReg - Ratio {1-ratio:.0%}:{ratio:.0%} - 30 Days - RMSE: {rmse2_30:.2f}, MAE: {mae2_30:.2f}, MAPE: {mape2_30:.2f}%')
        print(f'RandomForest - Ratio {1-ratio:.0%}:{ratio:.0%} - 60 Days - RMSE: {rmse1_60:.2f}, MAE: {mae1_60:.2f}, MAPE: {mape1_60:.2f}%')
        print(f'Bagging/LinearReg - Ratio {1-ratio:.0%}:{ratio:.0%} - 60 Days - RMSE: {rmse2_60:.2f}, MAE: {mae2_60:.2f}, MAPE: {mape2_60:.2f}%')
        print(f'RandomForest - Ratio {1-ratio:.0%}:{ratio:.0%} - 90 Days - RMSE: {rmse1_90:.2f}, MAE: {mae1_90:.2f}, MAPE: {mape1_90:.2f}%')
        print(f'Bagging/LinearReg - Ratio {1-ratio:.0%}:{ratio:.0%} - 90 Days - RMSE: {rmse2_90:.2f}, MAE: {mae2_90:.2f}, MAPE: {mape2_90:.2f}%')

        # Vẽ kết quả của mô hình RandomForest
        plot_results(train, test, preds1_30, preds1_60, preds1_90, dates_30, dates_60, dates_90, stock_name, 'RandomForest')

        # Vẽ kết quả của mô hình Bagging/LinearRegression
        plot_results(train, test, preds2_30, preds2_60, preds2_90, dates_30, dates_60, dates_90, stock_name, 'Bagging/LR')

# Tỉ lệ train:test
ratios = [0.2]

# Dự đoán cho từng mã cổ phiếu
process_stock('DataSBS.csv', 'SBS', ratios)

RandomForest - Ratio 80%:20% - 30 Days - RMSE: 1159.52, MAE: 1093.10, MAPE: 15.14%
Bagging/LinearReg - Ratio 80%:20% - 30 Days - RMSE: 4763.42, MAE: 4751.60, MAPE: 67.48%
RandomForest - Ratio 80%:20% - 60 Days - RMSE: 1771.36, MAE: 1762.29, MAPE: 22.67%
Bagging/LinearReg - Ratio 80%:20% - 60 Days - RMSE: 4152.53, MAE: 4149.32, MAPE: 53.53%
RandomForest - Ratio 80%:20% - 90 Days - RMSE: 2064.60, MAE: 2045.16, MAPE: 25.33%
Bagging/LinearReg - Ratio 80%:20% - 90 Days - RMSE: 3855.99, MAE: 3848.32, MAPE: 48.00%
